In [1]:
# Install required YOLO
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

# Load Data


In [3]:
!curl -L "https://drive.usercontent.google.com/download?id=1ft3vgar02Co1EfTRoD-5gQrxdK__JIjM&export=download&confirm=t&uuid=16e26256-f6e6-4f66-8fff-fb748da2027c" -o nuimages_yolo_filtered.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5043M  100 5043M    0     0  79.4M      0  0:01:03  0:01:03 --:--:-- 35.1M


In [6]:
!rm -rf sample_data
!unzip nuimages_yolo_filtered.zip
!unzip latest_runs.zip

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
  inflating: data/nuimages_yolo/val/images/c3c74b6c145e4447bae087fe2b1acc71.jpg  
  inflating: data/nuimages_yolo/val/images/5d1a223db4e8431999360dd7389ac6e2.jpg  
  inflating: data/nuimages_yolo/val/images/48e30057283e4da2b0e16e7327c8de0f.jpg  
  inflating: data/nuimages_yolo/val/images/308f0d850caa46c1ae1273969bffded1.jpg  
  inflating: data/nuimages_yolo/val/images/9e3ddda124ed4d15847eccebae78a223.jpg  
  inflating: data/nuimages_yolo/val/images/f23b1bfbcc1f49afba0056a4a647e2ae.jpg  
  inflating: data/nuimages_yolo/val/images/7663ce9899404eaca3d2b82fc0dcae3f.jpg  
  inflating: data/nuimages_yolo/val/images/b22e3639c0b247bda37e5ffabb066c70.jpg  
  inflating: data/nuimages_yolo/val/images/c7953af627404bb980943960bfc28bc7.jpg  
  inflating: data/nuimages_yolo/val/images/ba529d6480e04ec186589743c5dfd9c5.jpg  
  inflating: data/nuimages_yolo/val/images/80c5e63e10b34b5b954327457a8f5b00.jpg  
  inflating: data/nuimages_yolo

In [4]:
import os

os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import json
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from tqdm.notebook import tqdm
from PIL import Image
from pathlib import Path
import torch
import yaml

# Überprüfen der verfügbaren Hardware
print(f"PyTorch version: {torch.__version__}")
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

PyTorch version: 2.6.0+cu124
Using device: cuda


In [5]:
# Ausgabepfad für die YOLO-Trainingsdaten
output_path = "data/nuimages_yolo"
os.makedirs(output_path, exist_ok=True)

# Pfad zum vortrainierten YOLO-Modell
yolo_model_path = "yolo12n.pt"

## Model Training


In [7]:
# Val-Set verkleinern, damit es nicht so viel Zeit in Anspruch nimmt während Training
import os
import random
import shutil
from pathlib import Path

def reduce_val_dataset(
    val_dir,
    keep_ratio: float = 0.25,
    output_dir = None,
    seed: int = 42
) -> str:
    """
    Reduziert einen Validierungsdatensatz auf 25% durch zufällige Auswahl.

    Args:
        val_dir: Pfad zum Validierungsordner mit /images und /labels Unterordnern
        keep_ratio: Anteil der zu behaltenden Daten (Standard: 0.25 = 25%)
        output_dir: Ausgabepfad (wenn None, wird val_dir + "_reduced" verwendet)
        seed: Random seed für Reproduzierbarkeit

    Returns:
        Pfad zum reduzierten Validierungsdatensatz
    """
    random.seed(seed)

    # Pfade vorbereiten
    val_dir = Path(val_dir)
    output_dir = Path(output_dir) if output_dir else Path(f"{val_dir}_reduced")

    images_dir = val_dir / "images"
    labels_dir = val_dir / "labels"

    if not images_dir.exists() or not labels_dir.exists():
        raise ValueError(f"Die erwartete Ordnerstruktur mit /images und /labels wurde nicht gefunden in {val_dir}")

    # Ausgabeordner erstellen
    output_images_dir = output_dir / "images"
    output_labels_dir = output_dir / "labels"

    output_images_dir.mkdir(parents=True, exist_ok=True)
    output_labels_dir.mkdir(parents=True, exist_ok=True)

    # Alle Bild-IDs sammeln (ohne Dateiendung)
    image_files = list(images_dir.glob("*.jpg")) + list(images_dir.glob("*.jpeg")) + list(images_dir.glob("*.png"))
    image_ids = [img_path.stem for img_path in image_files]

    # Zufällig x% auswählen
    selected_ids = random.sample(image_ids, int(len(image_ids) * keep_ratio))

    # Ausgewählte Bilder und Labels kopieren
    for img_id in selected_ids:
        # Bild kopieren (suche nach allen unterstützten Dateiformaten)
        for ext in ['.jpg', '.jpeg', '.png']:
            img_path = images_dir / f"{img_id}{ext}"
            if img_path.exists():
                shutil.copy2(img_path, output_images_dir / f"{img_id}{ext}")
                break

        # Label kopieren
        label_path = labels_dir / f"{img_id}.txt"
        if label_path.exists():
            shutil.copy2(label_path, output_labels_dir / f"{img_id}.txt")

    print(f"Reduzierter Datensatz erstellt: {len(selected_ids)} von {len(image_ids)} Bildern ({keep_ratio*100:.1f}%)")
    print(f"Ausgabeordner: {output_dir}")

    return str(output_dir)

reduced_val_set_path = reduce_val_dataset(
  val_dir=str(Path(output_path) / "val"),
  output_dir="data/nuimages_yolo/val_reduced",
  keep_ratio=0.85
)

Reduzierter Datensatz erstellt: 5319 von 6258 Bildern (85.0%)
Ausgabeordner: data/nuimages_yolo/val_reduced


In [8]:
import yaml
from pathlib import Path

# Annahme: output_path und reduced_val_path sind bereits definiert
# output_path: Basis-Ordner mit der dataset.yaml
# reduced_val_path: Pfad zum neu erstellten reduzierten Validierungsdatensatz

# Pfad zur YAML-Datei
yaml_path = Path(output_path) / "dataset.yaml"

# YAML-Datei einlesen
with open(yaml_path, 'r') as file:
    data = yaml.safe_load(file)

# Vorherigen val-Pfad speichern
original_val_path = data['val']
print(f"Originaler Validierungspfad: {original_val_path}")

# Val-Pfad aktualisieren
data['val'] = reduced_val_set_path.split("/")[-1] + "/images"
print(f"Neuer Validierungspfad: {data['val']}")

# Aktualisierte YAML-Datei zurückschreiben
with open(yaml_path, 'w') as file:
    yaml.dump(data, file, default_flow_style=False, sort_keys=False)

print(f"Die Datei {yaml_path} wurde aktualisiert.")

# Backup erstellen (korrigierte Version)
backup_data = data.copy()
backup_data['val'] = original_val_path  # Original val-Pfad wiederherstellen

backup_path = Path(output_path) / "dataset_backup.yaml"
with open(backup_path, 'w') as file:
    yaml.dump(backup_data, file, default_flow_style=False, sort_keys=False)

print(f"Backup erstellt: {backup_path}")

Originaler Validierungspfad: val/images
Neuer Validierungspfad: val_reduced/images
Die Datei data/nuimages_yolo/dataset.yaml wurde aktualisiert.
Backup erstellt: data/nuimages_yolo/dataset_backup.yaml


In [ ]:
# !zip -r nuimages_yolo_val_red.zip data/nuimages_yolo

In [9]:
# !rm -rf runs
torch.cuda.empty_cache()

In [10]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import time

torch.cuda.empty_cache()

# 1. Modell laden (vortrainiertes YOLOv11n)
print(f"Lade vortrainiertes Modell: {yolo_model_path}")
model = YOLO(yolo_model_path)
model = YOLO("runs/detect/train/weights/last.pt")

# 2. Training konfigurieren und starten
print(f"Starte Training mit Gerät: {device}")

# Parameter für das Training
results = model.train(
    data=str(Path(output_path) / "dataset.yaml"),
    # Grundlegende Trainingsparameter
    epochs=55,  # Anzahl der Trainingszyklen
    imgsz=704,  # Bildgröße (Standard für YOLOv11n)
    device=device,  # Verwendetes Gerät (mps, cuda, cpu)
    verbose=True,
    batch=32, # ...adjusted
    close_mosaic=13,
    save_period=2,
    plots=True,
    scale=0.65,
    degrees=2,
    cos_lr=True,
    dropout=0.05,
    classes=range(1,6),
    resume=True
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Lade vortrainiertes Modell: yolo12n.pt


100%|██████████| 5.34M/5.34M [00:00<00:00, 107MB/s]


Starte Training mit Gerät: cuda
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=13, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=data/nuimages_yolo/dataset.yaml, degrees=2, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.05, dynamic=False, embed=None, epochs=55, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=704, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs/detect/train/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, proje

100%|██████████| 755k/755k [00:00<00:00, 22.3MB/s]


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              


...

                   all       5319      37118      0.798      0.627      0.729      0.476

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      55/55      9.12G       1.06     0.7039     0.9413         19        704: 100%|██████████| 796/796 [11:03<00:00,  1.20it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 84/84 [01:02<00:00,  1.34it/s]


                   all       5319      37118        0.8      0.626      0.729      0.476

8 epochs completed in 1.707 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 5.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.131 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv12n summary (fused): 159 layers, 2,557,898 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 84/84 [01:09<00:00,  1.22it/s]


                   all       5319      37118      0.798      0.627      0.729      0.476
                person       2462       7701      0.775      0.557      0.668      0.353
           vehicle.car       4597      19507      0.844      0.762      0.852      0.617
    vehicle.motorcycle        900       1130      0.787      0.643      0.738      0.456
           vehicle.bus        780        963      0.796      0.493      0.623      0.451
         vehicle.truck       2109       3483      0.763      0.549      0.657      0.458
           trafficcone       1150       4334      0.826      0.757      0.838      0.519
Speed: 0.2ms preprocess, 2.4ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs/detect/train


In [11]:
!rm -rf latest_runs.zip
!zip -r latest_runs.zip runs

  adding: runs/ (stored 0%)
  adding: runs/detect/ (stored 0%)
  adding: runs/detect/train/ (stored 0%)
  adding: runs/detect/train/train_batch33434.jpg (deflated 14%)
  adding: runs/detect/train/train_batch0.jpg (deflated 7%)
  adding: runs/detect/train/weights/ (stored 0%)
  adding: runs/detect/train/weights/epoch32.pt (deflated 10%)
  adding: runs/detect/train/weights/epoch2.pt (deflated 10%)
  adding: runs/detect/train/weights/epoch16.pt (deflated 10%)
  adding: runs/detect/train/weights/epoch28.pt (deflated 10%)
  adding: runs/detect/train/weights/epoch10.pt (deflated 10%)
  adding: runs/detect/train/weights/epoch24.pt (deflated 10%)
  adding: runs/detect/train/weights/epoch50.pt (deflated 10%)
  adding: runs/detect/train/weights/epoch46.pt (deflated 10%)
  adding: runs/detect/train/weights/epoch36.pt (deflated 10%)
  adding: runs/detect/train/weights/epoch12.pt (deflated 10%)
  adding: runs/detect/train/weights/epoch34.pt (deflated 10%)
  adding: runs/detect/train/weights/epoch14

In [12]:
model = YOLO("runs/detect/train/weights/best.pt")
model.val(data=str(backup_path), device=device, batch=32, plots=True, verbose=True)

YOLOv12n summary (fused): 159 layers, 2,557,898 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 38.6±8.9 MB/s, size: 191.6 KB)


val: Scanning /content/data/nuimages_yolo/val/labels... 6258 images, 259 backgrounds, 0 corrupt: 100%|██████████| 6258/6258 [00:18<00:00, 342.41it/s]


val: New cache created: /content/data/nuimages_yolo/val/labels.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 196/196 [01:23<00:00,  2.35it/s]


                   all       6258      44131      0.797      0.629       0.73      0.478
                person       2922       9153      0.774      0.558       0.67      0.356
           vehicle.car       5421      23219      0.841      0.763      0.853      0.616
    vehicle.motorcycle       1071       1348      0.781      0.642      0.732      0.452
           vehicle.bus        927       1150      0.798      0.506      0.633      0.463
         vehicle.truck       2512       4157       0.76      0.546      0.653      0.456
           trafficcone       1359       5104      0.826      0.757      0.837      0.522
Speed: 0.3ms preprocess, 3.7ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/val


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78c8cd55d090>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     

In [13]:
!zip -r train_704_yolo12.zip runs

  adding: runs/ (stored 0%)
  adding: runs/detect/ (stored 0%)
  adding: runs/detect/val/ (stored 0%)
  adding: runs/detect/val/val_batch0_pred.jpg (deflated 4%)
  adding: runs/detect/val/confusion_matrix.png (deflated 20%)
  adding: runs/detect/val/R_curve.png (deflated 7%)
  adding: runs/detect/val/val_batch0_labels.jpg (deflated 4%)
  adding: runs/detect/val/val_batch2_pred.jpg (deflated 4%)
  adding: runs/detect/val/PR_curve.png (deflated 7%)
  adding: runs/detect/val/F1_curve.png (deflated 7%)
  adding: runs/detect/val/confusion_matrix_normalized.png (deflated 20%)
  adding: runs/detect/val/val_batch1_pred.jpg (deflated 4%)
  adding: runs/detect/val/P_curve.png (deflated 9%)
  adding: runs/detect/val/val_batch2_labels.jpg (deflated 4%)
  adding: runs/detect/val/val_batch1_labels.jpg (deflated 4%)
  adding: runs/detect/train/ (stored 0%)
  adding: runs/detect/train/train_batch33434.jpg (deflated 14%)
  adding: runs/detect/train/train_batch0.jpg (deflated 7%)
  adding: runs/detect/t